### Evaluación de Chatbot y RAG

La Generación Aumentada por Recuperación (RAG) es una técnica que mejora los Modelos de Lenguaje Grandes (LLMs) al proporcionarles conocimiento externo relevante. Se ha convertido en uno de los enfoques más utilizados para construir aplicaciones de LLM.

Este tutorial te mostrará cómo evaluar tus aplicaciones RAG usando LangSmith. Aprenderás:

1. Cómo crear conjuntos de datos de prueba
2. Cómo ejecutar tu aplicación RAG sobre esos conjuntos de datos
3. Cómo medir el rendimiento de tu aplicación usando diferentes métricas de evaluación

#### Descripción General
Un flujo de trabajo típico de evaluación de RAG consiste en tres pasos principales:

1. Crear un conjunto de datos con preguntas y sus respuestas esperadas
2. Ejecutar tu aplicación RAG sobre esas preguntas
3. Usar evaluadores para medir qué tan bien funcionó tu aplicación, observando factores como:
 - Relevancia de la respuesta
 - Precisión de la respuesta
 - Calidad de la recuperación
 
Para este tutorial, crearemos y evaluaremos un bot que responde preguntas sobre algunas de las perspicaces publicaciones del blog de Lilian Weng.

### Evaluación de Chatbot

In [ ]:
# Importar el módulo os para interactuar con el sistema operativo
import os
# Importar load_dotenv para cargar variables de entorno desde un archivo .env
from dotenv import load_dotenv
# Cargar las variables de entorno desde el archivo .env al entorno del sistema
load_dotenv()

# Configurar la clave de API de LangSmith desde las variables de entorno
# LangSmith se usa para monitorear y evaluar aplicaciones LLM
os.environ["LANGSMITH_API_KEY"]=os.getenv("LANGSMITH_API_KEY")
# Configurar la clave de API de OpenAI desde las variables de entorno
# OpenAI proporciona los modelos de lenguaje que usaremos
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
# Activar el rastreo de LangSmith para monitorear todas las llamadas a la API
# Esto permite visualizar y analizar las interacciones con el modelo
os.environ["LANGSMITH_TRACING"]="true"

In [ ]:
# Importar el cliente de LangSmith para interactuar con la plataforma
from langsmith import Client

# Crear una instancia del cliente de LangSmith
# Este cliente nos permite crear datasets, ejemplos y ejecutar evaluaciones
client = Client()

# Definir el nombre del dataset: estos son tus casos de prueba
# Un dataset agrupa ejemplos de entrada/salida para evaluar el rendimiento del modelo
dataset_name = "Chatbots Evaluation"
# Crear un nuevo dataset en LangSmith con el nombre especificado
# Esto devuelve un objeto dataset con un ID único
dataset = client.create_dataset(dataset_name)
# Crear múltiples ejemplos dentro del dataset
# Cada ejemplo contiene:
# - inputs: La pregunta que se hará al modelo
# - outputs: La respuesta esperada/correcta (ground truth)
client.create_examples(
    # Especificar a qué dataset pertenecen estos ejemplos usando su ID
    dataset_id=dataset.id,
    # Lista de ejemplos con preguntas y respuestas esperadas
    examples=[
        {
            # Primera pregunta: sobre LangChain
            "inputs": {"question": "What is LangChain?"},
            # Respuesta esperada: definición concisa de LangChain
            "outputs": {"answer": "A framework for building LLM applications"},
        },
        {
            # Segunda pregunta: sobre LangSmith
            "inputs": {"question": "What is LangSmith?"},
            # Respuesta esperada: definición de LangSmith como plataforma de observación
            "outputs": {"answer": "A platform for observing and evaluating LLM applications"},
        },
        {
            # Tercera pregunta: sobre OpenAI
            "inputs": {"question": "What is OpenAI?"},
            # Respuesta esperada: OpenAI como empresa creadora de LLMs
            "outputs": {"answer": "A company that creates Large Language Models"},
        },
        {
            # Cuarta pregunta: sobre Google
            "inputs": {"question": "What is Google?"},
            # Respuesta esperada: Google como empresa tecnológica conocida por búsquedas
            "outputs": {"answer": "A technology company known for search"},
        },
        {
            # Quinta pregunta: sobre Mistral
            "inputs": {"question": "What is Mistral?"},
            # Respuesta esperada: Mistral como empresa creadora de LLMs
            "outputs": {"answer": "A company that creates Large Language Models"},
        }
    ]
)

### Definir Métricas (LLM como Juez)


In [ ]:
# Importar la biblioteca de OpenAI para interactuar con sus modelos
import openai
# Importar wrappers de langsmith para envolver el cliente de OpenAI
# Esto permite rastrear automáticamente todas las llamadas a OpenAI
from langsmith import wrappers
 
# Crear un cliente de OpenAI envuelto con funcionalidad de rastreo de LangSmith
# Esto registrará todas las interacciones con la API en LangSmith para análisis
openai_client=wrappers.wrap_openai(openai.OpenAI())

# Instrucciones del sistema para el LLM evaluador
# Define el rol del modelo como un profesor experto que califica respuestas
eval_instructions = "You are an expert professor specialized in grading students' answers to questions."

# Función evaluadora de corrección que compara respuesta predicha vs respuesta correcta
# Parámetros:
# - inputs: diccionario con la pregunta original
# - outputs: diccionario con la respuesta generada por el modelo
# - reference_outputs: diccionario con la respuesta correcta esperada
def correctness(inputs:dict,outputs:dict, reference_outputs:dict)->bool:
      # Construir el contenido del mensaje para el evaluador
      # Incluye la pregunta, respuesta correcta y respuesta del estudiante
      user_content = f"""You are grading the following question:
    {inputs['question']}
    Here is the real answer:
    {reference_outputs['answer']}
    You are grading the following predicted answer:
    {outputs['response']}
    Respond with CORRECT or INCORRECT:
    Grade:
    """
      # Llamar al modelo GPT-4o-mini para evaluar la corrección
      # temperature=0 hace que la respuesta sea determinística y consistente
      response=openai_client.chat.completions.create(
            # Usar el modelo mini para evaluaciones rápidas y económicas
            model="gpt-4o-mini",
            # Temperatura 0 para respuestas determinísticas
            temperature=0,
            # Mensajes que incluyen el rol del sistema y el contenido a evaluar
            messages=[
                  # Mensaje del sistema con las instrucciones del evaluador
                  {"role":"system","content":eval_instructions},
                  # Mensaje del usuario con la pregunta y respuestas a comparar
                  {"role":"user","content":user_content}
            ]
      # Extraer el contenido del mensaje de la primera opción de respuesta
      ).choices[0].message.content

      # Retornar True si la respuesta es "CORRECT", False en caso contrario
      return response == "CORRECT"

In [ ]:
# Métrica de Concisión - verifica si la respuesta generada no es excesivamente larga
# El objetivo es asegurar que el modelo genere respuestas concisas y no verbosas

# Función evaluadora de concisión
# Parámetros:
# - outputs: diccionario con la respuesta generada por el modelo
# - reference_outputs: diccionario con la respuesta de referencia esperada
def concision(outputs: dict, reference_outputs: dict) -> bool:
    # Comparar la longitud de la respuesta generada vs la esperada
    # La respuesta es considerada concisa si es menor a 2x la longitud de la referencia
    # len() cuenta el número de caracteres en cada string
    # Retorna 1 (True) si es concisa, 0 (False) si es demasiado larga
    return int(len(outputs["response"]) < 2 * len(reference_outputs["answer"]))

### Ejecutar Evaluaciones

In [ ]:
# Definir las instrucciones por defecto para el chatbot
# Estas instrucciones guían al modelo para que responda de manera concisa
default_instructions = "Respond to the users question in a short, concise manner (one short sentence)."

# Función principal de la aplicación del chatbot
# Esta función recibe una pregunta y genera una respuesta usando OpenAI
# Parámetros:
# - question: La pregunta del usuario (string)
# - model: El modelo de OpenAI a usar (default: "gpt-4o-mini")
# - instructions: Instrucciones del sistema para guiar al modelo
def my_app(question: str, model: str = "gpt-4o-mini", instructions: str = default_instructions) -> str:
    # Llamar a la API de OpenAI para generar una respuesta
    return openai_client.chat.completions.create(
        # Especificar qué modelo usar (gpt-4o-mini es rápido y económico)
        model=model,
        # Temperatura 0 para respuestas consistentes y determinísticas
        temperature=0,
        # Array de mensajes que define la conversación
        messages=[
            # Primer mensaje: instrucciones del sistema que definen el comportamiento del modelo
            {"role": "system", "content": instructions},
            # Segundo mensaje: la pregunta del usuario
            {"role": "user", "content": question},
        ],
    # Extraer el contenido de texto de la primera respuesta generada
    ).choices[0].message.content

In [ ]:
# Función objetivo para LangSmith - se ejecuta para cada punto de datos del dataset
# Esta función adapta la entrada del dataset al formato esperado por my_app
# y devuelve la salida en el formato esperado por los evaluadores

# Parámetros:
# - inputs: diccionario con la pregunta del dataset
def ls_target(inputs: str) -> dict:
    # Llamar a my_app con la pregunta extraída de inputs
    # Retornar un diccionario con clave "response" para que coincida con el formato esperado
    return {"response": my_app(inputs["question"])}

In [ ]:
# Ejecutar la evaluación completa del chatbot
# Esta función ejecuta el sistema de IA sobre todo el dataset y aplica los evaluadores
experiment_results=client.evaluate(
    # La función objetivo que se probará (nuestro sistema de IA)
    ls_target,
    # El nombre del dataset con los casos de prueba
    data=dataset_name,
    # Lista de funciones evaluadoras que medirán el rendimiento
    # correctness: verifica si la respuesta es correcta comparada con la referencia
    # concision: verifica si la respuesta es concisa (no más de 2x la longitud esperada)
    evaluators=[correctness,concision],
    # Prefijo para identificar este experimento en LangSmith
    # Útil para comparar diferentes configuraciones o modelos
    experiment_prefix="openai-4o-mini-chatbot"
)

In [ ]:
# Función objetivo modificada para probar un modelo diferente (GPT-4 Turbo)
# Esta versión usa el modelo más avanzado GPT-4 Turbo en lugar de GPT-4o-mini
# Permite comparar el rendimiento de diferentes modelos sobre el mismo dataset

# Parámetros:
# - inputs: diccionario con la pregunta del dataset
def ls_target(inputs: str) -> dict:
    # Llamar a my_app especificando explícitamente el modelo GPT-4 Turbo
    # GPT-4 Turbo es más capaz pero también más costoso que gpt-4o-mini
    return {"response": my_app(inputs["question"],model="gpt-4-turbo")}

In [ ]:
# Ejecutar una segunda evaluación usando GPT-4 Turbo
# Esto permite comparar directamente el rendimiento de GPT-4 Turbo vs GPT-4o-mini
experiment_results=client.evaluate(
    # La función objetivo con GPT-4 Turbo
    ls_target,
    # El mismo dataset de prueba para una comparación justa
    data=dataset_name,
    # Los mismos evaluadores para métricas consistentes
    evaluators=[correctness,concision],
    # Prefijo diferente para distinguir este experimento del anterior
    # LangSmith mostrará ambos experimentos lado a lado para comparación
    experiment_prefix="openai-4-turbo-chatbot"
)

### Evaluación para RAG

In [ ]:
# ===== CONFIGURACIÓN DEL SISTEMA RAG =====
# RAG: Retrieval-Augmented Generation (Generación Aumentada por Recuperación)

# Importar los componentes necesarios para construir el sistema RAG
from langchain_community.document_loaders import WebBaseLoader  # Cargador de documentos desde URLs
from langchain_core.vectorstores import InMemoryVectorStore  # Base de datos vectorial en memoria
from langchain_openai import OpenAIEmbeddings  # Embeddings de OpenAI para convertir texto a vectores
from langchain_text_splitters import RecursiveCharacterTextSplitter  # Divisor de texto inteligente

# Lista de URLs de los blogs de Lilian Weng que usaremos como base de conocimiento
# Estos artículos cubren temas sobre agentes, prompt engineering y ataques adversarios
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",  # Artículo sobre agentes de IA
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",  # Artículo sobre ingeniería de prompts
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",  # Artículo sobre ataques adversarios a LLMs
]

# Cargar los documentos desde cada URL usando WebBaseLoader
# List comprehension que itera sobre cada URL y carga su contenido
docs = [WebBaseLoader(url).load() for url in urls]
# Aplanar la lista de listas en una sola lista de documentos
# [item for sublist in docs for item in sublist] convierte [[doc1], [doc2], [doc3]] en [doc1, doc2, doc3]
docs_list = [item for sublist in docs for item in sublist]

# Inicializar un divisor de texto que respeta los tokens del modelo
# RecursiveCharacterTextSplitter divide el texto de manera inteligente
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    # Tamaño de cada chunk en tokens (250 tokens ≈ 187 palabras)
    chunk_size=250,
    # Sin superposición entre chunks (overlap=0)
    # Un overlap > 0 ayudaría a mantener contexto entre chunks
    chunk_overlap=0
)

# Dividir todos los documentos en chunks más pequeños
# Esto es crucial para RAG porque permite encontrar fragmentos específicos relevantes
doc_splits = text_splitter.split_documents(docs_list)

# Crear una base de datos vectorial en memoria con los chunks
# Los documentos se convierten a embeddings (vectores numéricos) usando OpenAIEmbeddings
vectorstore = InMemoryVectorStore.from_documents(
    # Los chunks de documentos que queremos almacenar
    documents=doc_splits,
    # El modelo de embedding que convierte texto a vectores
    # OpenAI usa text-embedding-ada-002 por defecto
    embedding=OpenAIEmbeddings(),
)

# Convertir el vectorstore en un retriever (recuperador)
# El retriever es responsable de buscar los documentos más relevantes
# k=6 significa que recuperará los 6 chunks más similares a la consulta
retriever = vectorstore.as_retriever(k=6)

In [ ]:
# Probar el retriever con una consulta de ejemplo
# Esta llamada busca los 6 documentos más relevantes sobre "agents" (agentes)
# El retriever usa similitud de coseno entre el embedding de la consulta y los embeddings de los documentos
retriever.invoke("what is agents")

In [ ]:
# Inicializar el modelo de lenguaje para el sistema RAG
# Usar init_chat_model permite inicializar modelos de diferentes proveedores con una sintaxis unificada
from langchain.chat_models import init_chat_model
# Inicializar el modelo GPT-4o-mini de OpenAI
# Formato: "proveedor:modelo"
llm=init_chat_model("openai:gpt-4o-mini")
# Mostrar la configuración del modelo
llm

In [ ]:
# Importar el decorador traceable para rastrear la ejecución en LangSmith
from langsmith import traceable

# Decorador @traceable() registra automáticamente cada llamada a esta función en LangSmith
# Esto permite monitorear el rendimiento, costos y comportamiento del sistema RAG
@traceable()
def rag_bot(question:str)->dict:
    """
    Bot RAG que responde preguntas usando documentos recuperados como contexto
    
    Proceso:
    1. Recuperar documentos relevantes usando el retriever
    2. Construir un prompt con los documentos como contexto
    3. Generar una respuesta usando el LLM
    """
    # PASO 1: Recuperar los documentos más relevantes para la pregunta
    # El retriever usa búsqueda de similitud semántica para encontrar los 6 chunks más relevantes
    docs=retriever.invoke(question)
    
    # PASO 2: Concatenar el contenido de todos los documentos recuperados en un solo string
    # Cada documento se une con un espacio para crear el contexto
    docs_string = " ".join(doc.page_content for doc in docs)

    # PASO 3: Construir las instrucciones del sistema con el contexto de los documentos
    # Estas instrucciones guían al modelo a responder basándose solo en los documentos proporcionados
    instructions = f"""You are a helpful assistant who is good at analyzing source information and answering questions.       Use the following source documents to answer the user's questions.       If you don't know the answer, just say that you don't know.       Use three sentences maximum and keep the answer concise.

Documents:
{docs_string}"""
    
    # PASO 4: Invocar el LLM con las instrucciones y la pregunta
    # El modelo genera una respuesta basándose en el contexto proporcionado
    ai_msg=llm.invoke([
         # Mensaje del sistema: instrucciones + contexto de documentos
         {"role": "system", "content": instructions},
         # Mensaje del usuario: la pregunta original
         {"role": "user", "content": question},
    ])
    
    # PASO 5: Retornar un diccionario con la respuesta y los documentos usados
    # Esto permite evaluar tanto la respuesta como la calidad de la recuperación
    return {"answer":ai_msg.content,"documents":docs}


In [ ]:
# Probar el bot RAG con una pregunta sobre agentes
# Esta llamada ejecutará todo el flujo: recuperación + generación
# El resultado incluirá tanto la respuesta como los documentos usados
rag_bot("What is agents")

### Dataset para Evaluación de RAG

In [ ]:
# Importar el cliente de LangSmith nuevamente
from langsmith import Client

# Crear una instancia del cliente
client=Client()

# Definir los ejemplos para el dataset de evaluación de RAG
# Estos ejemplos son preguntas específicas sobre el contenido de los blogs de Lilian Weng
# Cada ejemplo tiene una pregunta (input) y una respuesta esperada (output)
examples = [
    {
        # Pregunta 1: Sobre cómo los agentes ReAct usan la auto-reflexión
        "inputs": {"question": "How does the ReAct agent use self-reflection? "},
        # Respuesta esperada que describe la integración de razonamiento y acción
        "outputs": {"answer": "ReAct integrates reasoning and acting, performing actions - such tools like Wikipedia search API - and then observing / reasoning about the tool outputs."},
    },
    {
        # Pregunta 2: Sobre los tipos de sesgos en few-shot prompting
        "inputs": {"question": "What are the types of biases that can arise with few-shot prompting?"},
        # Respuesta esperada que enumera tres tipos de sesgos
        "outputs": {"answer": "The biases that can arise with few-shot prompting include (1) Majority label bias, (2) Recency bias, and (3) Common token bias."},
    },
    {
        # Pregunta 3: Sobre tipos de ataques adversarios
        "inputs": {"question": "What are five types of adversarial attacks?"},
        # Respuesta esperada que lista cinco tipos específicos de ataques
        "outputs": {"answer": "Five types of adversarial attacks are (1) Token manipulation, (2) Gradient based attack, (3) Jailbreak prompting, (4) Human red-teaming, (5) Model red-teaming."},
    }
]

# Crear un nuevo dataset específico para evaluar el sistema RAG
dataset_name="RAG Test Evaluation"
# Crear el dataset en LangSmith
dataset = client.create_dataset(dataset_name=dataset_name)
# Agregar los ejemplos al dataset
# Esto creará tres casos de prueba en LangSmith con IDs únicos
client.create_examples(
    # Asociar los ejemplos con el dataset creado
    dataset_id=dataset.id,
    # Los ejemplos definidos anteriormente
    examples=examples
)


### Evaluadores o Métricas para RAG
1. **Corrección (Correctness)**: Respuesta vs respuesta de referencia
- **Objetivo**: Medir "qué tan similar/correcta es la respuesta del RAG, en relación con una respuesta ground-truth"
- **Modo**: Requiere una respuesta ground truth (referencia) proporcionada a través de un dataset
- **Evaluador**: Usar LLM como juez para evaluar la corrección de la respuesta.

In [ ]:
# Importar tipos para definir el esquema de salida estructurada
from typing_extensions import Annotated,TypedDict

# ===== ESQUEMA DE SALIDA PARA CORRECCIÓN =====

# Definir el esquema de salida para la calificación de corrección
# TypedDict permite definir la estructura exacta que el LLM debe generar
class CorrectnessGrade(TypedDict):
    # NOTA: El orden en que se definen los campos es el orden en que el modelo los generará
    # Es útil poner explicaciones antes de respuestas porque obliga al modelo a pensar
    # antes de generar su respuesta final
    
    # Campo 1: Explicación del razonamiento (generado primero)
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    # Campo 2: Calificación booleana de corrección (generado después de la explicación)
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

# ===== INSTRUCCIONES PARA EL EVALUADOR DE CORRECCIÓN =====

# Prompt detallado para el LLM evaluador que actúa como profesor
correctness_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. 
(2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the student's answer meets all of the criteria.
A correctness value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Importar ChatOpenAI para crear el modelo evaluador
from langchain_openai import ChatOpenAI

# Crear un LLM evaluador con salida estructurada
# with_structured_output fuerza al modelo a generar JSON siguiendo el esquema CorrectnessGrade
grader_llm=ChatOpenAI(model="gpt-4o-mini",temperature=0).with_structured_output(
    CorrectnessGrade,  # El esquema que debe seguir la salida
    method="json_schema",  # Método de generación estructurada
    strict=True  # Modo estricto: la salida DEBE seguir exactamente el esquema
)

# ===== FUNCIÓN EVALUADORA DE CORRECCIÓN =====

# Función evaluadora para la precisión de respuestas RAG
def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """
    Evaluador para la precisión de respuestas RAG
    
    Parámetros:
    - inputs: diccionario con la pregunta original
    - outputs: diccionario con la respuesta generada por el RAG
    - reference_outputs: diccionario con la respuesta correcta esperada
    
    Retorna:
    - bool: True si la respuesta es correcta, False en caso contrario
    """
    # Construir el mensaje con la pregunta, respuesta correcta y respuesta del estudiante
    answers = f"""\
QUESTION: {inputs['question']}
GROUND TRUTH ANSWER: {reference_outputs['answer']}
STUDENT ANSWER: {outputs['answer']}"""

    # Ejecutar el evaluador LLM con las instrucciones y el contenido a evaluar
    grade = grader_llm.invoke([
        # Mensaje del sistema con instrucciones de cómo calificar
        {"role": "system", "content": correctness_instructions}, 
        # Mensaje del usuario con la pregunta y respuestas a comparar
        {"role": "user", "content": answers}
    ])
    
    # Retornar solo el valor booleano de corrección
    # grade es un diccionario con estructura {"explanation": "...", "correct": True/False}
    return grade["correct"]


### Relevancia: Respuesta vs entrada
El flujo es similar al anterior, pero simplemente observamos las entradas y salidas sin necesitar las reference_outputs. Sin una respuesta de referencia no podemos calificar la precisión, pero aún podemos calificar la relevancia—es decir, si el modelo abordó la pregunta del usuario o no.

In [ ]:
# ===== EVALUADOR DE RELEVANCIA =====
# Este evaluador mide si la respuesta es relevante para la pregunta
# No requiere una respuesta de referencia, solo evalúa si se abordó la pregunta

# Esquema de salida para la calificación de relevancia
class RelevanceGrade(TypedDict):
    # Explicación del razonamiento (generada primero para forzar al modelo a pensar)
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    # Calificación booleana de relevancia
    relevant: Annotated[bool, ..., "Provide the score on whether the answer addresses the question"]

# Instrucciones del prompt para evaluar relevancia
relevance_instructions="""You are a teacher grading a quiz. 

You will be given a QUESTION and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is concise and relevant to the QUESTION
(2) Ensure the STUDENT ANSWER helps to answer the QUESTION

Relevance:
A relevance value of True means that the student's answer meets all of the criteria.
A relevance value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# LLM evaluador de relevancia con salida estructurada
# Usar GPT-4o para evaluaciones más precisas de relevancia
relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
    RelevanceGrade,  # Esquema de salida
    method="json_schema",  # Método de estructuración
    strict=True  # Modo estricto
)

# Función evaluadora de relevancia
def relevance(inputs: dict, outputs: dict) -> bool:
    """
    Evaluador simple para la utilidad de respuestas RAG
    
    Parámetros:
    - inputs: diccionario con la pregunta original
    - outputs: diccionario con la respuesta generada
    
    Retorna:
    - bool: True si la respuesta es relevante, False en caso contrario
    """
    # Construir el mensaje con la pregunta y la respuesta del estudiante
    answer = f"QUESTION: {inputs['question']}\nSTUDENT ANSWER: {outputs['answer']}"
    
    # Invocar el LLM evaluador
    grade = relevance_llm.invoke([
        # Instrucciones del sistema
        {"role": "system", "content": relevance_instructions}, 
        # Contenido a evaluar
        {"role": "user", "content": answer}
    ])
    
    # Retornar el valor booleano de relevancia
    return grade["relevant"]

### Fundamentación (Groundedness): Respuesta vs documentos recuperados
Otra forma útil de evaluar respuestas sin necesitar respuestas de referencia es verificar si la respuesta está justificada por (o "fundamentada en") los documentos recuperados.

In [ ]:
# ===== EVALUADOR DE FUNDAMENTACIÓN (GROUNDEDNESS) =====
# Este evaluador verifica si la respuesta está basada en los documentos recuperados
# Detecta "alucinaciones" donde el modelo inventa información no presente en los documentos

# Esquema de salida para la calificación de fundamentación
class GroundedGrade(TypedDict):
    # Explicación del razonamiento
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    # Calificación booleana: True si está fundamentada, False si alucina
    grounded: Annotated[bool, ..., "Provide the score on if the answer hallucinates from the documents"]

# Instrucciones del prompt para evaluar fundamentación
grounded_instructions = """You are a teacher grading a quiz. 

You will be given FACTS and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is grounded in the FACTS. 
(2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# LLM evaluador de fundamentación con salida estructurada
grounded_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
    GroundedGrade,  # Esquema de salida
    method="json_schema",
    strict=True
)

# Función evaluadora de fundamentación
def groundedness(inputs: dict, outputs: dict) -> bool:
    """
    Evaluador simple para la fundamentación de respuestas RAG
    
    Verifica si la respuesta está basada en los documentos recuperados
    o si el modelo está "alucinando" información no presente en ellos
    
    Parámetros:
    - inputs: diccionario con la pregunta (no usado en esta evaluación)
    - outputs: diccionario con la respuesta y los documentos recuperados
    
    Retorna:
    - bool: True si está fundamentada, False si alucina
    """
    # Concatenar el contenido de todos los documentos recuperados
    # Estos son los "FACTS" que el modelo debería haber usado
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    
    # Construir el mensaje con los hechos y la respuesta del estudiante
    answer = f"FACTS: {doc_string}\nSTUDENT ANSWER: {outputs['answer']}"
    
    # Invocar el LLM evaluador
    grade = grounded_llm.invoke([
        {"role": "system", "content": grounded_instructions},
        {"role": "user", "content": answer}
    ])
    
    # Retornar el valor booleano de fundamentación
    return grade["grounded"]

### Relevancia de Recuperación: Documentos recuperados vs entrada

In [ ]:
# ===== EVALUADOR DE RELEVANCIA DE RECUPERACIÓN =====
# Este evaluador mide si los documentos recuperados son relevantes para la pregunta
# Es crucial para evaluar la calidad del componente de recuperación del sistema RAG

# Esquema de salida para la calificación de relevancia de recuperación
class RetrievalRelevanceGrade(TypedDict):
    # Explicación del razonamiento
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    # Calificación booleana: True si los documentos son relevantes
    relevant: Annotated[bool, ..., "True if the retrieved documents are relevant to the question, False otherwise"]

# Instrucciones del prompt para evaluar relevancia de recuperación
retrieval_relevance_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION and a set of FACTS provided by the student. 

Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# LLM evaluador de relevancia de recuperación con salida estructurada
retrieval_relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(
    RetrievalRelevanceGrade,  # Esquema de salida
    method="json_schema",
    strict=True
)

# Función evaluadora de relevancia de recuperación
def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
    """
    Evaluador para la relevancia de documentos recuperados
    
    Mide si el retriever está recuperando documentos relevantes
    para responder la pregunta del usuario
    
    Parámetros:
    - inputs: diccionario con la pregunta original
    - outputs: diccionario con los documentos recuperados
    
    Retorna:
    - bool: True si los documentos son relevantes, False si no lo son
    """
    # Concatenar el contenido de todos los documentos recuperados
    # Estos son los "FACTS" que el retriever encontró
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    
    # Construir el mensaje con los hechos y la pregunta
    answer = f"FACTS: {doc_string}\nQUESTION: {inputs['question']}"

    # Ejecutar el evaluador LLM
    grade = retrieval_relevance_llm.invoke([
        # Instrucciones del sistema
        {"role": "system", "content": retrieval_relevance_instructions}, 
        # Contenido a evaluar
        {"role": "user", "content": answer}
    ])
    
    # Retornar el valor booleano de relevancia
    return grade["relevant"]

### Ejecutar la evaluación completa del sistema RAG

In [ ]:
# ===== EJECUTAR EVALUACIÓN COMPLETA DEL SISTEMA RAG =====

# Función objetivo que envuelve el bot RAG para la evaluación
def target(inputs: dict) -> dict:
    """
    Función wrapper que adapta rag_bot al formato esperado por client.evaluate
    
    Parámetros:
    - inputs: diccionario con la pregunta del dataset
    
    Retorna:
    - dict: respuesta y documentos del bot RAG
    """
    return rag_bot(inputs["question"])

# Ejecutar la evaluación completa con múltiples evaluadores
experiment_results = client.evaluate(
    # La función objetivo (sistema RAG) que se evaluará
    target,
    # El dataset de prueba con preguntas y respuestas esperadas
    data=dataset_name,
    # Lista de CUATRO evaluadores que miden diferentes aspectos:
    # 1. correctness: ¿La respuesta es correcta vs la referencia?
    # 2. groundedness: ¿La respuesta está basada en los documentos recuperados?
    # 3. relevance: ¿La respuesta es relevante para la pregunta?
    # 4. retrieval_relevance: ¿Los documentos recuperados son relevantes?
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    # Prefijo para identificar este experimento en LangSmith
    experiment_prefix="rag-doc-relevance",
    # Metadatos adicionales para documentar la configuración del experimento
    metadata={"version": "LCEL context, gpt-4-0125-preview"},
)

# Convertir los resultados a un DataFrame de pandas para análisis local
# Esto permite visualizar y analizar los resultados directamente en el notebook
experiment_results.to_pandas()